01-initial_data_clean.ipynb

first run download_convert.sh to download sql file and convert to csv
- Expand predicates with OR operations into individual predicates
- Convert cuis that are entrez ids into cuis
- Change neg props to the same prop with a negative flag
- Make a separate nodes table

In [1]:
import os
import pickle
%matplotlib inline
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from collections import defaultdict
import itertools
import numpy as np

In [32]:
# read in csv, group rows together with the same subj -> pred -> obj and keep count of number of pmids
# done in chunks
cols = ['SUBJECT_CUI','PREDICATE','OBJECT_CUI','PMID']
gb_cols = ['SUBJECT_CUI','PREDICATE','OBJECT_CUI']
sem_df = pd.DataFrame(columns=cols)
df_iter = pd.read_csv('semmedVER31_R_PREDICATION_to12312017.csv', dtype=str, usecols=cols, chunksize=10000000)
for chunk in tqdm_notebook(df_iter, total=10):
    chunk.PMID = chunk.PMID.astype("str")
    c = chunk.groupby(gb_cols).PMID.agg(";".join).reset_index()
    sem_df = sem_df.append(c)

In [39]:
# group everything together again
print(len(sem_df))
sem_df = sem_df.groupby(gb_cols).PMID.agg(";".join).reset_index()
print(len(sem_df))

19476062
19476062


In [40]:
print(sem_df.PMID.str.count(";").value_counts()[:5])

0    13036630
1     2739873
2     1095890
3      592645
4      369792
Name: PMID, dtype: int64


In [41]:
sem_df.head()

,SUBJECT_CUI,PREDICATE,OBJECT_CUI,PMID
0,1,AFFECTS,C0020291,6298464
1,1,AFFECTS,C0023946,20801151
2,1,AFFECTS,C0028754,19789049
3,1,AFFECTS,C0597304,1409557
4,1,AFFECTS,C0599816,7617239


In [42]:
sem_df.to_csv("edges1.csv")

#### in semmedDB some subjects and objects of extracted statments contained the pipe character | as an indicator of multiple concepts in the sentence.

In [43]:
# separate out lines with pipes from the rest
multi_start = sem_df['SUBJECT_CUI'].str.contains('|', regex=False)
multi_end = sem_df['OBJECT_CUI'].str.contains('|', regex=False)
pipe_lines = sem_df[multi_start | multi_end].copy()
sem_df = sem_df[~multi_start & ~multi_end]
print('Rows with multiple subjects or objects {:,}'.format(len(pipe_lines)))
print('Rows with only 1 subject AND only 1 object {:,}'.format(len(sem_df)))

Rows with multiple subjects or objects 2,681,265
Rows with only 1 subject AND only 1 object 16,794,797


In [44]:
pipe_lines.SUBJECT_CUI = pipe_lines.SUBJECT_CUI.str.split('|')
pipe_lines.OBJECT_CUI = pipe_lines.OBJECT_CUI.str.split('|')
pipe_lines.head()

,SUBJECT_CUI,PREDICATE,OBJECT_CUI,PMID
23,[1],COEXISTS_WITH,"[C0003241, 4099]",24639825;24639825;24639825
41,[1],INTERACTS_WITH,"[C1333653, 9846]",22865653
63,[1],STIMULATES,"[C0164786, 207]",22865653
71,[1],compared_with,"[C0040643, 866, 57733]",28919732
77,[10],compared_with,"[C0796518, 10]",8631131


In [45]:
# do the combinations
lines = []
for row in tqdm_notebook(pipe_lines.itertuples(), total=len(pipe_lines)):
    #print(row)
    a = [row.SUBJECT_CUI, row.OBJECT_CUI]
    c = list(itertools.product(*a))
    lines.extend([{'SUBJECT_CUI':x[0], 'PREDICATE':row.PREDICATE, 'OBJECT_CUI':x[1], 'PMID': row.PMID} for x in c])
expanded_df = pd.DataFrame(lines)

In [46]:
print(len(expanded_df))
expanded_df.head()

6453951


,OBJECT_CUI,PMID,PREDICATE,SUBJECT_CUI
0,C0003241,24639825;24639825;24639825,COEXISTS_WITH,1
1,4099,24639825;24639825;24639825,COEXISTS_WITH,1
2,C1333653,22865653,INTERACTS_WITH,1
3,9846,22865653,INTERACTS_WITH,1
4,C0164786,22865653,STIMULATES,1


In [47]:
# append the expanded rows to the original df
print(len(sem_df))
sem_df = sem_df.append(expanded_df, sort=True)
print(len(sem_df))

16794797
23248748


In [48]:
# Group rows together again and aggregate pmids
sem_df = sem_df.groupby(gb_cols).PMID.agg(";".join).reset_index()
print(len(sem_df))

22322647


### There are many CUIs that are not actually cuis. Mike has determined they are (usually/always?) entrez IDs

In [49]:
# are going to get them out of this MRSAT file (download from umls metathersourosuous)
# I did: cat MRSAT.RRF.a* | gzip > MRSAT.RRF.gz
names = list("abcdefghijklmn")
iter_csv = pd.read_csv("MRSAT.RRF.gz", delimiter="|", names=names, index_col=None, chunksize=1000000)
chunks = []
umls_entrez = dict()
for chunk in tqdm_notebook(iter_csv, total=67668372/1000000):
    chunk.fillna(method='ffill', inplace=True)
    chunk = chunk[chunk.i == "ENTREZGENE_ID"]
    d = dict(zip(chunk.a, chunk.k))
    umls_entrez.update(d)

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2963: DtypeWarning: Columns (7,10) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2963: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [50]:
entrez_umls = {v:k for k,v in umls_entrez.items()}
entrez_umls['9987']

'C1415639'

In [51]:
sem_df.SUBJECT_CUI = sem_df.SUBJECT_CUI.map(lambda x:entrez_umls[x] if x in entrez_umls else x)
sem_df.OBJECT_CUI = sem_df.OBJECT_CUI.map(lambda x:entrez_umls[x] if x in entrez_umls else x)

In [52]:
noncdf = sem_df[~sem_df.SUBJECT_CUI.str.startswith("C")]
print(len(noncdf))

21694


In [53]:
# dump everything that doesn't starts with a "C"
print(len(sem_df))
sem_df = sem_df[sem_df.SUBJECT_CUI.str.startswith("C")]
sem_df = sem_df[sem_df.OBJECT_CUI.str.startswith("C")]
print(len(sem_df))

22322647
22280924


In [3]:
# sem_df.to_csv("edges2.csv")
# sem_df = pd.read_csv("edges2.csv", index_col=0)

/usr/local/lib/python3.5/dist-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
# change the neg to the same prop without neg and add a neg column# chang 
idx = sem_df["PREDICATE"].str.startswith("NEG_")
sem_df['NEG'] = False
sem_df.loc[idx, 'NEG'] = True
sem_df.loc[idx, 'PREDICATE'] = sem_df[idx].PREDICATE.str.replace("NEG_", "")
sem_df[sem_df.NEG].head()

,SUBJECT_CUI,PREDICATE,OBJECT_CUI,PMID,NEG
436,C1412045,AFFECTS,C0031845,19737390,True
437,C1412045,ASSOCIATED_WITH,C0206754,28792692,True
438,C1412045,CAUSES,C1457887,18943647,True
439,C1412045,TREATS,C0001807,1616455,True
855,C0796518,AFFECTS,C0242379,15808403,True


In [5]:
sem_df.to_csv("edges3.csv")

In [6]:
### Make a nodes table
conso = pd.read_csv("MRCONSO_ENG.RRF.gz", delimiter="|", index_col=None, names = list("abcdefghijklmnopqrs"))
conso = conso[(conso['c'] == "P") & (conso['e'] == "PF")]
conso.head()

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (9,10,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s
0,C0000005,ENG,P,L0000005,PF,S0007492,Y,A26634265,NaN,M0019694,D012711,MSH,PEP,D012711,(131)I-Macroaggregated Albumin,0,N,256.0,NaN
2,C0000039,ENG,P,L0000039,PF,S17175117,N,A28315139,9194921.0,1926948,NaN,RXNORM,IN,1926948,"1,2-dipalmitoylphosphatidylcholine",0,N,256.0,NaN
3,C0000039,ENG,P,L0000039,PF,S17175117,Y,A28572604,NaN,NaN,NaN,MTH,PN,NOCODE,"1,2-dipalmitoylphosphatidylcholine",0,N,256.0,NaN
32,C0000052,ENG,P,L0000052,PF,S0007584,N,A0016535,NaN,M0023173,D015061,MSH,MH,D015061,"1,4-alpha-Glucan Branching Enzyme",0,N,256.0,NaN
33,C0000052,ENG,P,L0000052,PF,S0007584,N,A18020417,NaN,N0000168345,NaN,NDFRT,PT,N0000168345,"1,4-alpha-Glucan Branching Enzyme",0,N,256.0,NaN


In [7]:
node_label = dict(zip(conso.a, conso.o))
print(len(node_label))

3654581


In [8]:
nodes = set(sem_df.SUBJECT_CUI) | set(sem_df.OBJECT_CUI)
print(len(nodes))

271725


In [9]:
nodes = pd.DataFrame({"ID":x, "LABEL": node_label.get(x)} for x in nodes)
nodes = nodes.dropna()
print(len(nodes))
nodes.head()

259227


,ID,LABEL
0,C0061133,gastrin releasing peptide (14-27)
1,C1523610,"regulation of tube length, open tracheal system"
2,C0312636,Antibody to hepatitis E virus
3,C1532578,mL/cm H2O
4,C0539817,cytochrome p30


In [10]:
nodes.to_csv("nodes1.csv")